### Initializing pymc3

In [343]:
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt
import math
from pymc3 import sample_ppc
from pymc3.plots import plot_posterior
from scipy.stats import multivariate_normal, poisson, norm, invgamma
from seaborn import kdeplot

print('Running on PyMC3 v{}'.format(pm.__version__))

Running on PyMC3 v3.3


### Specifying training data

In [344]:
#X Data
trt = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

base = np.array([11, 11, 6, 8, 66, 27, 12, 52, 23, 10, 52, 33, 18,
                 42, 87, 50, 18, 111, 18, 20, 12, 9, 17, 28, 55, 9, 10, 47, 76, 38, 19, 10,
                 19, 24, 31, 14, 11, 67, 41, 7, 22, 13, 46, 36, 38, 7, 36, 11, 151, 22, 41,
                 32, 56, 24, 16, 22, 25, 13, 12])
age = np.array([31, 30, 25, 36, 22, 29, 31, 42, 37, 28, 36, 24, 23,
                36, 26, 26, 28, 31, 32,21, 29, 21, 32, 25, 30, 40,
                19, 22, 18, 32, 20, 30, 18, 24, 30, 35, 27, 20,22,
                28, 23, 40, 33, 21, 35, 25, 26, 25, 22, 32, 25,35, 21, 41, 32, 26, 21, 36, 37])

v4 = np.array([0,0,0,1])

#Y data
y = np.array([[5,3,3,3],[3,5,3,3],[2,4,0,5],[4,4,1,4],[7,18,9,21],[5,2,8,7],[6,4,0,2],[40,20,21,12],[5,6,6,5],
   [14,13,6,0],[26,12,6,22],[12,6,8,4],[4,4,6,2],[7,9,12,14],[16,24,10,9],[11,0,0,5],[0,0,3,3],[37,29,28,29],
   [3,5,2,5],[3,0,6,7],[3,4,3,4],[3,4,3,4],[2,3,3,5],[8,12,2,8],[18,24,76,25],[2,1,2,1],[3,1,4,2],
   [13,15,13,12],[11,14,9,8],[8,7,9,4],[0,4,3,0],[3,6,1,3],[2,6,7,4],[4,3,1,3],[22,17,19,16],
   [5,4,7,4],[2,4,0,4],[3,7,7,7],[4,18,2,5],[2,1,1,0],[0,2,4,0],[5,4,0,3],[11,14,25,15],[10,5,3,8],
   [19,7,6,7],[1,1,2,3],[6,10,8,8],[2,1,0,0],[102,65,72,63],[4,3,2,4],[8,6,5,7],[1,3,1,5],[18,11,28,13],
   [6,3,4,0],[3,5,4,3],[1,23,19,8],[2,3,0,1],[0,0,0,0],[1,4,3,2]])

data_size = len(age)
days = 4

### Preprocessing

In [345]:
#augment data to have same dimensions
trt = trt.reshape(-1,1)
trt = np.tile(trt,(1,4))

base = base.reshape(-1,1)
base = np.tile(base,(1,4))

age = age.reshape(-1,1)
age = np.tile(age,(1,4))

v4 = v4.reshape(-1,1)
v4 = np.tile(v4,(1,59)).T

print(trt.shape,base.shape,age.shape,v4.shape)

(59, 4) (59, 4) (59, 4) (59, 4)


### Specifying the model

In [346]:
#Creating the model

model = pm.Model()

with model:
    #priors
    
    var_b  = pm.InverseGamma('var_b',1,1)
    var_b1 = pm.InverseGamma('var_b1',1,1)
    
    a_base = pm.Normal('alpha_base',mu=0,sd=100)
    a_trt = pm.Normal('alpha_trt',mu=0,sd=100)
    a_bt = pm.Normal('alpha_bt',mu=0,sd=100)
    a_age = pm.Normal('alpha_age',mu=0,sd=100)
    a_v4 = pm.Normal('alpha_v4',mu=0,sd=100)
    a_not = pm.Normal('alpha_not',mu=0,sd=100)
    
    b1 = pm.Normal('b1',mu=0,sd=np.sqrt(var_b1))
    b = pm.Normal('b',mu=0,sd=np.sqrt(var_b))
    
    #expected value of parameter mu
   # log_mu = a_not + (a_base * np.log(base/4)) + (a_trt * trt) + (a_bt * trt * np.log(base/4)) \
   #             + (a_age * np.log(age)) + b1
    log_mu = a_not + (a_v4*v4) + (a_base * np.log(base/4)) + (a_trt * trt) + (a_bt * trt * np.log(base/4)) + (a_age * np.log(age)) + b1 + b

    mu = np.exp(log_mu)
    #likelihood
    Y_obs = pm.Poisson('Y_obs',mu,observed=y)

### Run built in MCMC

In [ ]:
#get the posterior 
with model:
    step = pm.Metropolis()
    trace = pm.sample(500,step)

In [ ]:
pm.summary(trace)

In [ ]:
_ = pm.traceplot(trace)

In [ ]:
ppc = sample_ppc(trace, model=model, samples=500)

In [ ]:
predictions = ppc['Y_obs']

#get the mean and variance of the predictions
mean_preds = np.mean(predictions,axis=0)
var_preds =  np.var(predictions,axis=0)

mse_loss = np.sum((y - mean_preds) ** 2) / (59 * 4.0)

print('MSE loss: ',mse_loss)
print('True Values: ',y)
print('Mean predictions: ',mean_preds)
print('Variance of predictions: ',var_preds)

## Own Implementation

In [ ]:
def find_likelihood(theta):
    
    a_not = theta[0]
    a_base = theta[1]
    a_trt = theta[2]
    a_bt = theta[3]
    a_age = theta[4]
    a_v4 = theta[5]
    var_b1 = abs(theta[6])
    var_b = abs(theta[7])
    
    b1 = norm.rvs(0,np.sqrt(var_b1),size=1)
    b = norm.rvs(0,np.sqrt(var_b),size=1)
    
    log_mu = a_not + (a_v4*v4) + (a_base* np.log(base/4)) + (a_trt * trt) + (a_bt * trt * np.log(base/4)) + (a_age * np.log(age)) + b1 + b
    mu = np.exp(log_mu)
    
    mu = mu.flatten()
    ypred = y.flatten()
    #print(mu,ypred)
    #print(poisson.pmf(ypred,mu).shape)
    likelihood  = np.sum(poisson.pmf(ypred,mu))
    
    return likelihood

def find_prior(theta,prior_params):
    
    a_not = theta[0]
    a_base = theta[1]
    a_trt = theta[2]
    a_bt = theta[3]
    a_age = theta[4]
    a_v4 = theta[5]
    var_b1 = theta[6]
    var_b = theta[7]
    
    prior_anot = norm(prior_params['a_mu'],prior_params['a_sd']).pdf(a_not)
    prior_abase = norm(prior_params['a_mu'],prior_params['a_sd']).pdf(a_base)
    prior_atrt = norm(prior_params['a_mu'],prior_params['a_sd']).pdf(a_trt)
    prior_abt = norm(prior_params['a_mu'],prior_params['a_sd']).pdf(a_bt)
    prior_age = norm(prior_params['a_mu'],prior_params['a_sd']).pdf(a_age)
    prior_av4 = norm(prior_params['a_mu'],prior_params['a_sd']).pdf(a_v4)
    
    prior_varb1 = invgamma(a=prior_params['var_b'],scale=prior_params['var_b']).pdf(var_b1)
    prior_varb = invgamma(a=prior_params['var_b'],scale=prior_params['var_b']).pdf(var_b)
    
    cumulative_prior = prior_anot * prior_abase * prior_atrt * prior_abt * \
            prior_age * prior_av4 * prior_varb * prior_varb1
    #print(prior_b,prior_b1)
    return cumulative_prior
    
    

In [ ]:
def tune(scale, acc_rate):
    """
    Tunes the scaling parameter for the proposal distribution
    according to the acceptance rate over the last tune_interval:
    Rate    Variance adaptation
    ----    -------------------
    <0.001        x 0.1
    <0.05         x 0.5
    <0.2          x 0.9
    >0.5          x 1.1
    >0.75         x 2
    >0.95         x 10
    """

    # Switch statement
    if acc_rate < 0.001:
        # reduce by 90 percent
        scale *= 0.1
    elif acc_rate < 0.05:
        # reduce by 50 percent
        scale *= 0.5
    elif acc_rate < 0.2:
        # reduce by ten percent
        scale *= 0.9
    elif acc_rate > 0.95:
        # increase by factor of ten
        scale *= 10.0
    elif acc_rate > 0.75:
        # increase by double
        scale *= 2
    elif acc_rate > 0.5:
        # increase by ten percent
        scale *= 1.1

    return scale

In [ ]:
def mcmc_mh(prior_params,samples=10,interval=10,proposal_width=1):
    
    #initial parameters
    #in this order : a_not,a_base,a_trt,a_bt,a_age,a_v4,b1,b,var_b1,var_b
    theta_current = np.ones(8)
    #print(theta_current)
    posterior = [theta_current]
    
    
    accepted = 0
    
    interval_size = interval
    
    for i in range(samples-1):
        
        if i % interval_size == 0:
            proposal_width = tune(proposal_width,accepted/float(interval_size))
            accepted = 0
            print('proposal width: ',proposal_width)
        
        #proposal parameters
        theta_proposal = multivariate_normal(mean=theta_current,cov=proposal_width).rvs()

        #find likelihoods
        likelihood_current = find_likelihood(theta_current)
        likelihood_proposal = find_likelihood(theta_proposal)
        
        #prior probability of current and proposed
        prior_current = find_prior(theta_current,prior_params)
        prior_proposal = find_prior(theta_proposal,prior_params)
        
        # p(z hat)
        p_current = likelihood_current * prior_current
        p_proposal = likelihood_proposal * prior_proposal
        
        p_accept = p_proposal / p_current
        print(p_accept)
        accept = np.random.rand() < p_accept
        
        if accept: 
            theta_current = theta_proposal
            accepted += 1
            #print('changed')
            
        posterior.append(theta_current)
        
        
    posterior = np.array(posterior)    
    return posterior

In [ ]:
prior_params = {
    'a_mu' : 0,
    'a_sd' : 100,
    'var_b' : 1
}

my_trace = mcmc_mh(prior_params,interval=100,samples=500)
print(my_trace.mean(axis=0))

In [ ]:
def plot(trace):
    f, axarr = plt.subplots(len(trace.keys()), 2,figsize=(15,30))
    for i,key in enumerate(trace.keys()):
        values = trace[key]
        kdeplot(values, ax=axarr[i,0])
        axarr[i,0].set_title(key)
        axarr[i,0].set_ylabel('Frequency')
        axarr[i,1].plot(values)
        axarr[i,1].set_title(key)
        axarr[i,1].set_ylabel('Sample value')
        

In [ ]:
trace_dict = {}
trace_dict['alpha_not'] = my_trace[:,0]
trace_dict['alpha_base'] = my_trace[:,1]
trace_dict['alpha_trt'] = my_trace[:,2]
trace_dict['alpha_age'] = my_trace[:,3]
trace_dict['alpha_bt'] = my_trace[:,4]
trace_dict['alpha_v4'] = my_trace[:,5]
trace_dict['var_b1'] = my_trace[:,6]
trace_dict['var_b'] = my_trace[:,7]

plot(trace_dict)

In [ ]:
def predict(theta):
    a_not = theta[0]
    a_base = theta[1]
    a_trt = theta[2]
    a_bt = theta[3]
    a_age = theta[4]
    a_v4 = theta[5]
    var_b1 = abs(theta[6])
    var_b = abs(theta[7])
    
    b1 = norm.rvs(0,np.sqrt(var_b1),size=1)
    b = norm.rvs(0,np.sqrt(var_b),size=1)
    
    log_mu = a_not + (a_v4*v4) + (a_base* np.log(base/4)) + (a_trt * trt) + (a_bt * trt * np.log(base/4)) + (a_age * np.log(age)) + b1 + b
    mu = np.exp(log_mu)
    
    pred = poisson.rvs(mu)
    return pred

def get_ppc(trace,samples=50,size=10):
    
    #sample parameters from the trace
    trace_len = trace.shape[0]
    rand_indexes = np.random.randint(0,trace_len,size=samples)
    
    #sampled parameters
    params = trace[rand_indexes]
    
    #predictions 
    predictions = []
    for p in params:
        #generate 'size' samples for each parameter setting:
        for i in range(size):
            y = predict(p)
            predictions.append(y)
    
    predictions = np.array(predictions)
    return predictions

In [ ]:
preds = get_ppc(my_trace)
preds.shape

In [ ]:
preds.mean(axis=0)